In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time
import json

# Cargar clave desde .env
load_dotenv()
API_KEY = os.getenv("alpha_key")
if not API_KEY:
    raise SystemExit("❌ No se encontró ALPHA_KEY en el .env")

# Lista de tickers
tickers_nasdaq = ['NVDA','MSFT','AAPL']

# Fecha límite (últimos 3 meses)
hoy = datetime.today()
fecha_limite = hoy - timedelta(days=90)

def descargar_eod(symbol):
    """Descarga datos diarios de Alpha Vantage y filtra últimos 3 meses"""
    url = (
        f"https://www.alphavantage.co/query?"
        f"function=TIME_SERIES_DAILY&symbol={symbol}" #si se paga se puede poner TIME_SERIES_DAILY_ADJUST
        f"&outputsize=full&apikey={API_KEY}"
    )
    
    try:
        r = requests.get(url, timeout=10)
        data = r.json()
        
        # DEBUGGING: Imprimir la respuesta completa
        print(f"\n📋 Respuesta de API para {symbol}:")
        print(json.dumps(data, indent=2)[:500])  # Primeros 500 caracteres
        
        # Verificar si hay mensaje de error
        if "Error Message" in data:
            print(f"❌ {symbol}: {data['Error Message']}")
            return None
        
        # Verificar si hay mensaje de límite alcanzado
        if "Information" in data:
            print(f"⚠️ {symbol}: {data['Information']}")
            return None
            
        # Verificar si hay mensaje de nota
        if "Note" in data:
            print(f"⚠️ {symbol}: Límite de API alcanzado - {data['Note']}")
            return None
        
        # Buscar la clave de Time Series
        key = next((k for k in data.keys() if "Time Series" in k), None)
        if not key:
            print(f"⚠️ {symbol}: No se encontró 'Time Series' en la respuesta")
            print(f"   Claves disponibles: {list(data.keys())}")
            return None
        
        df = pd.DataFrame.from_dict(data[key], orient='index', dtype=float)
        df.index = pd.to_datetime(df.index)
        df = df.rename(columns={
            '1. open':'open',
            '2. high':'high',
            '3. low':'low',
            '4. close':'close',
            #'5. adjusted close':'adj_close',-----> parece que no me deja obtener el adj Close desde el tier gratuito. (activar si se paga)
            '5. volume':'volume'
        })
        df = df[df.index >= fecha_limite]
        df['symbol'] = symbol
        
        print(f"✅ {symbol}: {len(df)} filas descargadas")
        return df.reset_index().rename(columns={'index': 'date'})
        
    except requests.exceptions.RequestException as e:
        print(f"❌ {symbol}: Error de conexión - {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ {symbol}: Error al parsear JSON - {e}")
        return None

# Descargar para cada ticker
todo = []
for i, ticker in enumerate(tickers_nasdaq):
    print(f"\n[{i+1}/{len(tickers_nasdaq)}] Descargando {ticker}...")
    df = descargar_eod(ticker)
    if df is not None:
        todo.append(df)
    
    # Esperar 12 segundos entre requests (5 por minuto = 1 cada 12 seg)
    if i < len(tickers_nasdaq) - 1:  # No esperar después del último
        print(f"⏳ Esperando 12 segundos...")
        time.sleep(12)

# Combinar todo en un solo DataFrame
if todo:
    df_all = pd.concat(todo, ignore_index=True)
    df_all = df_all.sort_values(['symbol', 'date'])
    df_all.to_csv("nasdaq_eod_3meses.csv", index=False)
    print(f"\n✅ Datos guardados en nasdaq_eod_3meses.csv ({len(df_all)} filas).")
else:
    print("\n❌ No se pudo descargar ningún dato.")


SystemExit: ❌ No se encontró ALPHA_KEY en el .env

/Users/emiliosanchez/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
